## - sampler = DWaveSampler
## - composite = EmbeddingComposite

In [1]:
#!pip3 install dwave-system

In [3]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
import itertools
import random
import matplotlib.pyplot as plt
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave.embedding.chain_strength import uniform_torque_compensation
import dimod
import timeit

In [4]:
def calc_marginals(df):                   
    return np.array([                      
        sum(df['Y']), #0
        np.dot(df['Y'], df['SEX']),      #1
        np.dot(df['Y'], df['AOP']),    #2  
    ])     

In [5]:
def make_Hamiltonian(df):
    t_list = calc_marginals(df)
    N=len(df)
    dup_list = [(i, i) for i in range(N)]
    comb_list = [(i, j) for i in range(N) for j in range(i+1, N)]
    
    lin_Y = [1-2*t_list[0] for (i, _) in dup_list] #同じy同士
    quad_Y = [2 for (i, j) in comb_list] #異なるy同士
    num_Y = t_list[0]**2 #数字の二乗
    
    SEX = df['SEX'].iloc
    lin_SEX  = [(SEX[i] - 2 * t_list[1]) * SEX[i] for (i, _) in dup_list]
    quad_SEX  = [2*SEX[i] * SEX[j] for (i, j) in comb_list]
    num_SEX  = t_list[1]**2
    
    AOP = df['AOP'].iloc
    lin_AOP = [(AOP[i] - 2 * t_list[2]) * AOP[i] for (i, _) in dup_list]
    quad_AOP = [2*AOP[i] * AOP[j] for (i, j) in comb_list]
    num_AOP = t_list[2]**2
    
    #lin
    lin_list = [sum(lin) for lin in zip(lin_Y, lin_SEX, lin_AOP)]
    lin = {i: lin_list[i] for (i, _) in dup_list}
    
    #quad
    quad_values = [sum(quad) for quad in zip(quad_Y, quad_SEX, quad_AOP)]
    quad = {ij: quad_values[n] for (n, ij) in enumerate(comb_list)}
    
    #num
    num = num_Y + num_SEX + num_AOP
    
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY)#dic, dic, num

In [6]:
def find_valid_y_1(df, num_reads):                                                        
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    
    #bqm
    bqm = make_Hamiltonian(df)

    #method
    res = qa_sampler.sample(bqm, chain_break_fraction=True, num_reads=num_reads)
    print(res)
    
    valid_y_list = []                                                           
    valid_y_num = 0                                                             
    for y_info in list(res.record):
        if y_info[1] == 0.:
            valid_y_num += 1                                        
            valid_y_list.append(list(y_info[0])) 
                                                      
    return valid_y_list, valid_y_num       

In [7]:
def find_valid_y_2(df, num_reads):                                                        
    DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = DWAVE_TOKEN
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
                                                                                                                                                                                                                                               
    #bqm
    bqm = make_Hamiltonian(df)

    #method
    res = qa_sampler.sample(bqm, chain_break_fraction=False, num_reads=num_reads)
    print(res)
    
    valid_y_list = []                                                           
    valid_y_num = 0                                                             
    for y_info in list(res.record):
        if y_info[1] == 0.0:
            valid_y_num += 1                                        
            valid_y_list.append(list(y_info[0]))
                                                      
    return valid_y_list, valid_y_num

In [8]:
def find_valid_y_3(df, num_reads, max_chain_strength, chain_break_fraction):                                                        
    DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = DWAVE_TOKEN
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    valid_y_list = []                                                           
    valid_y_num = 0     
    
    bqm = make_Hamiltonian(df)

    for chain_strength in range(0, max_chain_strength):
        res = qa_sampler.sample(
            bqm, chain_strength = chain_strength, 
            chain_break_fraction=chain_break_fraction, num_reads=num_reads)
        print(res)
                                                                   
        for y_info in list(res.record):
            if y_info[1] == 0.:
                valid_y = list(y_info[0])
                if len(valid_y_list)==0:
                    valid_y_num += 1                                        
                    valid_y_list.append(valid_y)

                elif all([valid_y != p for p in valid_y_list]):
                    valid_y_num += 1                                    
                    valid_y_list.append(valid_y)
                                                      
    return valid_y_list, valid_y_num

In [9]:
def find_valid_y_4(df, num_reads, max_chain_strength, chain_break_fraction):                                                        
    DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = DWAVE_TOKEN
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    valid_y_list = {}                                                                   
    valid_y_num = {}
    
    bqm = make_Hamiltonian(df)

    for chain_strength in range(0, max_chain_strength):
        res = qa_sampler.sample(
            bqm, chain_strength = chain_strength, 
            chain_break_fraction=chain_break_fraction, num_reads=num_reads)
        print(res)
        
        valid_y_list[chain_strength] = []                                                           
        valid_y_num[chain_strength] = 0                                                             
        for y_info in list(res.record):
            if y_info[1] == 0.0:
                valid_y = list(y_info[0])
                if len(valid_y_list)==0:
                    valid_y_num[chain_strength] += 1                                        
                    valid_y_list[chain_strength].append(valid_y)

                elif all([valid_y != p for p in valid_y_list[chain_strength]]):
                    valid_y_num[chain_strength] += 1                                    
                    valid_y_list[chain_strength].append(valid_y)
                                                      
    return valid_y_list, valid_y_num

### [qa_sampler.sampleのパラメータ](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/generated/dwave.system.composites.AutoEmbeddingComposite.sample.html)
- chain_strength(float)
- chain_break_fraction(bool):Add a chain_break_fraction field to the unembedded dimod.SampleSet with the fraction of chains broken before unembedding.
- num_reads
- return_embedding(bool):詳細を表示するか否か

### 16bitで確認

In [10]:
df = pd.read_csv('../../input/ost16.csv', sep=',',index_col=0)

In [11]:
calc_marginals(df)#t1 = 6

array([8, 4, 4])

In [12]:
#find_valid_y_1(df, num_reads=100)

In [13]:
#find_valid_y_2(df, num_reads=100)

### ここまでのまとめ
- chain_break_fraction=False

In [14]:
find_valid_y_3(df, num_reads=100, max_chain_strength=10, chain_break_fraction=True)

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
55  1  1  0  1  0  0  0  1  0  0  0  0  1  1  1  1    0.0       1  0.9375
62  1  1  0  1  0  0  1  1  0  0  1  0  0  1  0  1    1.0       1     1.0
64  1  1  0  0  0  0  0  1  0  1  0  0  1  1  1  1    1.0       1     1.0
66  1  1  0  0  1  1  0  0  0  1  0  0  1  1  0  1    1.0       1  0.9375
71  1  1  0  0  1  1  0  0  0  1  0  0  1  1  0  1    1.0       1  0.9375
73  0  0  0  1  0  0  1  1  1  0  1  1  0  1  0  1    1.0       1     1.0
4   1  1  1  1  0  0  0  1  0  1  0  0  0  1  0  1    2.0       1  0.9375
5   1  1  1  1  0  0  0  1  0  1  0  0  0  1  0  1    2.0       1  0.9375
6   1  1  1  1  0  0  0  1  0  1  0  0  0  1  0  1    2.0       1  0.9375
7   1  1  1  1  0  0  0  1  0  1  0  0  0  1  0  1    2.0       1  0.9375
8   1  1  1  1  0  0  0  1  0  1  0  0  0  1  0  1    2.0       1  0.9375
9   1  1  1  1  0  0  0  1  0  1  0  0  0  1  0  1    2.0       1   0.875
22  1  1  1  1  0  0  0  1  0  1  0  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  0  0  0  1  0  1  0  0  1  1  1  0  1  0  1    0.0      33   0.625
6   1  0  0  0  1  0  1  0  0  1  1  1  0  1  0  1    0.0       3   0.625
7   1  0  0  0  1  0  1  0  0  1  1  1  0  1  0  1    0.0       2   0.625
13  1  0  0  0  1  0  1  0  0  1  1  1  0  1  0  1    0.0       1   0.625
17  1  0  0  0  1  0  1  0  0  1  1  1  0  1  0  1    0.0       1  0.6875
1   1  0  0  0  0  0  1  0  0  1  1  1  0  1  0  1    3.0       8   0.625
3   1  0  0  0  0  0  1  0  0  1  1  1  0  1  0  1    3.0       4   0.625
4   1  0  0  0  0  0  1  0  0  1  1  1  0  1  0  1    3.0      12   0.625
8   1  0  0  0  0  0  1  0  0  1  1  1  0  1  0  1    3.0       1   0.625
10  1  0  0  0  0  0  1  0  0  1  1  1  0  1  0  1    3.0       1   0.625
11  1  0  0  0  0  0  1  0  0  1  1  1  0  1  0  1    3.0       1   0.625
12  1  0  0  0  0  0  1  0  0  1  1  1  0  1  0  1    3.0       1   0.625
14  1  0  0  0  0  0  1  0  0  1  1  1

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
4   1  0  0  0  1  0  0  1  1  0  1  1  0  1  0  1    1.0       4   0.125
5   1  0  0  0  1  0  0  1  1  0  1  0  1  1  0  1    1.0       9   0.125
11  1  0  0  0  1  0  0  1  1  0  1  1  0  1  0  1    1.0       2  0.1875
16  1  0  0  0  1  0  0  1  1  0  1  0  1  1  0  1    1.0       1   0.125
17  1  0  0  0  1  0  0  0  1  0  1  0  1  1  0  1    1.0       1   0.125
18  1  0  0  0  1  0  0  1  1  0  1  1  0  1  0  1    1.0       2   0.125
21  1  0  0  0  1  0  0  1  1  0  1  1  0  1  0  1    1.0       3  0.1875
22  1  0  0  0  1  0  0  1  1  0  1  1  0  1  0  1    1.0       1  0.1875
23  1  0  0  0  1  0  0  1  1  0  1  0  1  1  0  1    1.0       2  0.1875
29  1  0  0  0  0  0  0  1  1  0  1  1  1  1  0  1    1.0       1  0.0625
32  1  0  1  0  0  1  1  1  0  0  1  0  0  1  1  0    1.0       1     0.0
33  1  0  0  0  1  0  0  1  1  0  1  1  0  1  0  1    1.0       1  0.1875
2   1  0  0  0  1  0  0  1  1  0  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   0  0  0  1  1  0  1  0  0  1  1  0  0  1  1  1    0.0       1     0.0
1   1  0  0  0  0  1  0  1  1  1  0  0  0  1  1  1    0.0       1     0.0
2   1  1  0  0  0  0  1  1  0  1  0  0  1  1  1  0    0.0       1     0.0
3   1  0  0  0  1  1  1  0  0  0  1  0  1  1  0  1    1.0       1     0.0
4   0  0  0  0  0  1  1  1  1  0  0  1  0  1  1  1    1.0       1     0.0
5   1  1  0  0  1  1  1  0  0  0  0  0  1  1  0  1    1.0       1     0.0
6   1  0  0  0  0  0  0  1  0  1  1  1  1  0  1  1    1.0       1     0.0
7   1  1  0  0  0  1  1  0  0  1  0  0  1  1  0  0    1.0       1     0.0
8   0  0  0  1  0  1  0  0  1  0  1  0  0  1  1  1    1.0       1     0.0
9   0  1  0  0  1  1  1  0  0  0  0  0  1  1  0  1    1.0       2     0.0
10  0  0  0  0  1  0  0  1  0  1  0  1  1  1  1  1    1.0       1     0.0
11  0  0  0  1  0  0  0  0  1  0  1  1  0  1  1  1    1.0       1     0.0
12  1  1  1  0  0  0  1  0  0  0  0  1

([[1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1],
  [1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1],
  [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1],
  [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1],
  [1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
  [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
  [1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
  [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1],
  [1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0],
  [1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
  [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1],
  [0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1],
  [1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]],
 16)

In [15]:
find_valid_y_3(df, num_reads=100, max_chain_strength=10, chain_break_fraction=False)

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   1  1  0  0  0  0  1  0  1  0  1  1  0  0  1  1    1.0       4
1   1  1  0  0  0  0  1  0  1  0  1  1  0  0  0  1    1.0       3
2   1  1  0  0  0  0  1  0  1  0  1  1  0  0  1  1    1.0       2
5   1  1  0  0  0  0  1  0  1  0  1  1  0  0  1  1    1.0       1
6   1  1  0  0  0  0  1  0  1  0  1  1  0  0  1  1    1.0       1
7   0  1  0  0  0  0  1  0  1  0  1  1  0  0  1  1    1.0       2
8   1  1  0  1  1  0  0  0  0  0  1  0  1  1  0  1    1.0       1
9   1  1  0  1  1  0  0  0  0  0  1  0  1  1  0  1    1.0       1
14  1  1  0  0  0  0  1  0  1  0  1  1  0  0  1  1    1.0       1
17  0  1  0  0  0  0  1  0  1  0  1  1  0  0  1  1    1.0       1
18  1  1  0  0  0  0  1  0  1  0  1  1  0  0  1  1    1.0       1
20  1  1  0  0  0  0  1  0  1  0  1  1  0  0  1  1    1.0       1
21  1  1  0  0  0  0  1  0  1  0  1  1  0  0  0  1    1.0       1
29  1  1  0  0  0  0  1  0  1  0  1  1  0  0  1  1    1.0       1
35  1  1  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   0  0  1  1  1  0  1  0  0  0  1  0  0  1  1  1    1.0      32
1   0  0  1  1  1  0  1  0  0  0  1  0  0  1  1  1    1.0      32
2   0  0  1  1  1  0  1  0  0  0  1  0  0  1  1  1    1.0       9
3   0  0  1  1  1  0  1  0  0  0  1  0  0  1  1  1    1.0       1
5   0  0  1  1  1  0  1  0  0  0  1  0  0  1  1  1    1.0       3
6   0  0  1  1  1  0  1  0  0  0  1  0  0  1  1  1    1.0       1
9   0  0  1  1  1  0  1  0  0  0  1  0  0  1  1  1    1.0       1
10  0  0  1  1  1  0  1  0  0  0  1  0  0  1  1  1    1.0       2
11  0  0  1  1  1  0  1  0  0  0  1  0  0  1  1  1    1.0       1
12  1  1  0  1  0  1  0  1  1  1  0  0  0  0  0  0    5.0       4
13  1  1  0  1  0  1  0  1  1  1  0  0  0  0  0  0    5.0       2
14  1  1  0  1  0  1  0  1  1  1  0  0  0  0  0  0    5.0       1
15  1  1  0  1  0  1  0  1  1  1  0  0  0  0  0  0    5.0       1
4   0  0  1  1  1  0  1  0  0  0  1  0  1  1  1  1    6.0       4
7   0  0  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   0  1  1  0  1  0  1  0  0  1  0  0  0  1  1  1    0.0       1
1   0  0  1  0  0  1  1  1  0  1  0  0  0  1  1  1    0.0       1
2   0  1  1  0  0  1  1  0  0  1  0  0  0  1  1  1    0.0       1
3   0  0  1  0  0  0  1  0  0  1  1  1  0  1  1  1    0.0       1
6   1  0  1  0  1  0  1  1  0  1  0  0  0  1  0  1    0.0       1
7   0  0  1  0  1  0  1  0  0  1  0  1  0  1  1  1    1.0       1
8   0  1  1  0  0  1  1  0  1  0  0  0  0  1  1  1    1.0       1
9   0  1  1  0  1  1  1  0  0  0  0  0  0  1  1  1    1.0       1
10  0  0  1  0  0  1  1  0  1  1  0  0  0  1  1  1    1.0       1
11  0  0  1  0  1  1  1  0  0  1  0  0  0  1  1  1    1.0       2
12  0  1  1  1  0  1  1  0  0  0  0  0  0  1  1  1    1.0       1
13  0  0  1  0  0  1  1  0  1  0  1  0  0  1  1  1    1.0       1
14  0  0  1  0  1  1  1  0  0  0  1  0  0  1  1  1    1.0       1
15  1  0  0  0  0  0  0  1  0  1  0  1  1  1  1  0    1.0       1
17  0  0  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   1  1  0  1  0  0  0  1  0  0  0  0  1  1  1  1    0.0       1
1   1  1  0  1  1  0  0  1  0  0  0  0  0  1  1  1    0.0       1
2   0  1  0  1  0  0  1  1  0  0  0  1  0  1  1  1    0.0       1
3   1  1  0  1  0  0  0  0  0  0  1  1  0  1  1  1    0.0       1
4   1  1  1  0  1  0  0  1  0  0  0  0  0  1  1  1    0.0       1
5   1  1  0  0  1  1  1  1  0  0  0  0  0  1  0  1    0.0       1
6   1  1  0  0  1  0  1  1  0  0  0  1  0  1  0  1    0.0       1
7   1  0  0  0  0  1  1  1  0  0  1  1  0  1  0  1    0.0       1
8   0  1  0  1  0  1  1  0  0  0  0  0  1  1  1  1    1.0       1
9   1  1  0  1  1  0  0  0  0  0  1  0  0  0  1  1    1.0       1
10  1  0  0  1  0  1  0  1  0  0  0  0  1  1  1  1    1.0       1
11  0  1  0  1  0  0  1  1  0  0  1  0  0  1  1  1    1.0       1
12  1  1  0  0  0  1  1  1  1  0  0  0  0  0  1  1    1.0       1
13  1  1  0  0  0  1  1  1  0  1  0  0  0  0  1  0    1.0       1
14  1  1  

([[0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1],
  [0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1],
  [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1],
  [0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1],
  [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
  [0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1],
  [1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1],
  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
  [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1],
  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
  [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1],
  [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
  [1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1],
  [1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1],
  [0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1],
  [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1],
  [1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
  [1, 1, 0, 

In [16]:
find_valid_y_3(df, num_reads=100, max_chain_strength=50, chain_break_fraction=True)

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       4     1.0
1   1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       5     1.0
2   1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       1     1.0
3   1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       1  0.9375
4   1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       2  0.9375
5   1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       1     1.0
7   1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       1     1.0
11  1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       2     1.0
24  1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       1     1.0
25  1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       2  0.9375
26  1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       1  0.9375
30  1  1  0  1  0  0  0  1  0  1  0  0  1  1  1  0    1.0       2     1.0
31  1  1  0  1  0  0  0  1  0  1  0  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0      12  0.4375
1   1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       6     0.5
2   1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       3   0.375
3   1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       6     0.5
4   1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       4  0.4375
6   1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       1   0.375
7   1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       3  0.5625
8   1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       2   0.375
10  1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       2  0.4375
11  1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       1  0.4375
12  1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       3     0.5
13  1  0  0  0  0  0  1  0  1  1  1  0  1  1  1  0    1.0       1  0.4375
14  1  0  0  0  0  0  1  0  1  1  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  0  1  0  0  1  1  1  0  1  0  0  0  1  0  1    0.0       1     0.0
1   1  1  1  0  0  0  1  0  0  1  1  0  0  1  1  0    0.0       1     0.0
2   0  1  1  0  0  0  1  0  1  0  1  0  0  1  1  1    0.0       1     0.0
3   1  0  1  0  0  0  1  0  0  1  1  1  0  1  0  1    0.0       1     0.0
4   1  1  0  0  0  0  0  0  1  0  0  1  1  1  1  1    1.0       1     0.0
5   1  1  1  0  0  0  0  0  0  1  1  1  0  1  1  0    1.0       1     0.0
6   1  0  1  0  0  0  0  1  0  1  1  1  0  1  1  0    1.0       1     0.0
7   1  1  1  0  0  0  1  1  0  1  0  1  0  1  0  0    1.0       1     0.0
8   0  1  0  0  0  0  1  1  1  0  0  0  1  0  1  1    1.0       1     0.0
9   1  0  0  0  0  0  1  1  1  1  1  0  0  0  1  0    1.0       1     0.0
10  1  0  1  0  0  0  0  0  0  1  1  1  0  1  1  0    1.0       1     0.0
11  1  1  0  0  1  0  1  0  0  1  1  0  0  0  1  0    1.0       1     0.0
12  1  0  0  0  1  0  0  1  0  1  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  0  0  0  0  0  0  1  0  1  0  1  1  1  1  1    0.0       2     0.0
1   1  1  0  0  0  0  1  1  0  1  0  1  0  0  1  1    0.0       1     0.0
2   1  1  0  0  0  0  1  1  0  0  0  1  1  1  0  1    0.0       1     0.0
3   1  0  0  0  0  1  1  1  0  0  1  1  0  1  0  1    0.0       1     0.0
4   1  0  0  0  0  0  1  1  0  1  0  1  1  1  0  1    0.0       1     0.0
5   1  0  0  0  0  1  1  1  0  1  0  1  0  1  0  1    0.0       1     0.0
6   1  0  0  0  0  0  1  1  1  1  0  1  0  1  0  1    0.0       1     0.0
7   1  1  1  0  1  0  0  1  0  0  0  0  0  1  1  1    0.0       1     0.0
8   0  0  0  0  0  0  1  1  0  1  0  1  1  1  1  1    0.0       1     0.0
9   1  0  1  0  0  0  1  1  0  1  1  0  0  1  1  0    0.0       1     0.0
10  1  1  0  0  0  0  1  0  0  1  0  1  1  1  0  1    0.0       1     0.0
11  1  1  0  0  0  0  1  0  1  1  0  1  0  1  0  1    0.0       1     0.0
12  1  1  0  1  0  0  1  0  0  1  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   0  0  1  1  0  0  1  1  0  0  1  0  0  1  1  1    0.0       1     0.0
1   1  0  0  0  1  0  1  1  0  1  0  1  0  1  0  1    0.0       1     0.0
2   0  0  0  1  0  1  0  1  1  0  1  0  0  1  1  1    1.0       1     0.0
3   0  0  0  0  0  1  1  1  1  0  1  0  0  1  1  0    1.0       1     0.0
4   0  0  1  1  0  1  0  0  0  0  1  0  0  1  1  1    1.0       1     0.0
5   0  0  0  0  1  0  1  1  0  1  1  0  1  1  1  0    1.0       1     0.0
6   1  0  1  1  0  1  0  0  0  1  1  0  0  1  0  1    1.0       1     0.0
7   1  1  0  0  1  0  1  1  0  0  1  0  1  1  0  0    1.0       1     0.0
8   0  1  0  1  0  1  0  0  0  0  1  1  0  1  1  1    1.0       1     0.0
9   1  0  0  0  1  0  1  0  0  1  1  0  1  1  1  0    1.0       1     0.0
10  0  0  1  1  0  1  1  0  0  0  1  0  0  1  0  1    1.0       1     0.0
11  0  1  0  0  1  0  1  0  0  0  1  1  1  1  0  1    1.0       1     0.0
12  0  0  1  0  0  1  1  0  1  1  1  0

...

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  0  1  0  1  0  0  0  0  1  1  0  0  0  1  1    1.0       1     0.0
1   0  0  0  1  1  0  1  1  0  1  0  0  0  1  1  0    1.0       1     0.0
2   1  1  0  0  1  0  1  0  1  1  1  0  0  0  0  1    1.0       1     0.0
3   1  1  0  1  0  0  1  0  1  1  0  0  0  1  0  0    1.0       1     0.0
4   1  1  0  1  1  0  0  1  0  0  1  0  0  0  1  1    1.0       1     0.0
5   1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1     0.0
6   0  0  0  1  1  1  0  0  0  1  1  0  0  1  1  0    2.0       1     0.0
7   1  0  1  1  0  0  0  0  0  1  1  0  1  0  1  1    2.0       1     0.0
8   0  0  0  1  0  0  1  0  1  0  1  0  1  0  1  1    2.0       1     0.0
9   1  0  0  0  1  0  0  1  0  0  1  1  1  0  1  1    2.0       1     0.0
10  1  0  1  0  1  0  1  0  0  1  0  0  1  1  0  0    2.0       1     0.0
11  0  0  0  0  1  1  1  0  1  1  1  0  0  0  0  1    2.0       1     0.0
12  1  0  1  1  1  0  0  0  0  0  0  1

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  1  0  0  0  0  1  1  1  1  0  0  0  0  1  1    0.0       1     0.0
1   1  1  0  0  0  1  1  1  0  0  1  0  0  1  1  0    0.0       1     0.0
2   1  1  0  0  0  1  1  0  0  0  1  0  1  1  0  1    0.0       1     0.0
3   1  1  0  0  0  1  0  1  1  0  0  0  0  1  1  1    0.0       1     0.0
4   1  1  0  0  0  1  1  0  0  0  1  0  1  1  0  0    1.0       1     0.0
5   1  1  1  1  0  0  0  1  0  0  0  0  0  0  1  1    1.0       1     0.0
6   0  1  1  1  0  0  1  1  0  0  0  0  0  1  1  0    1.0       1     0.0
7   0  0  1  1  0  0  1  1  0  1  0  0  1  1  0  1    1.0       1     0.0
8   1  1  1  0  0  1  0  1  0  0  1  0  0  1  1  0    1.0       1     0.0
9   0  0  0  0  0  0  1  1  1  1  1  0  0  1  1  1    1.0       1     0.0
10  1  0  0  0  0  1  0  0  0  1  1  1  1  1  0  1    1.0       1     0.0
11  1  0  0  0  0  1  0  0  1  1  1  0  1  1  0  1    1.0       1     0.0
12  0  0  0  0  0  1  0  0  0  1  1  1

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  1  0  0  0  0  1  1  0  1  0  1  0  1  1  0    0.0       1     0.0
1   1  1  0  0  0  0  1  0  0  0  1  1  1  0  1  1    1.0       1     0.0
2   0  0  1  0  0  0  1  1  1  0  1  1  0  1  0  1    1.0       1     0.0
3   1  0  0  1  1  0  0  1  0  1  0  0  1  1  0  1    1.0       1     0.0
4   1  0  0  0  0  0  0  1  0  0  1  1  1  1  1  0    1.0       1     0.0
5   1  0  0  1  0  0  0  1  1  1  0  0  1  1  0  1    1.0       1     0.0
6   1  0  1  0  0  0  1  1  0  1  0  0  1  0  1  1    1.0       1     0.0
7   1  0  1  0  0  0  1  1  0  1  1  0  0  1  1  1    1.0       1     0.0
8   1  0  1  0  0  0  1  1  0  1  1  0  0  0  1  0    1.0       1     0.0
9   0  0  0  0  1  0  1  1  0  1  1  1  0  0  1  1    1.0       1     0.0
10  1  0  1  0  1  0  0  1  0  1  0  1  0  1  0  1    1.0       1     0.0
11  0  0  0  1  1  0  1  1  0  0  0  1  1  1  0  1    2.0       1     0.0
12  1  0  1  1  0  0  0  1  0  0  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   0  1  0  1  0  0  0  0  1  0  0  1  0  1  1  1    1.0       1     0.0
1   0  0  1  0  0  0  1  1  1  0  0  0  1  1  1  1    1.0       1     0.0
2   0  1  1  0  0  0  1  0  1  0  0  1  0  1  1  1    1.0       1     0.0
3   0  0  0  0  1  1  1  0  1  0  1  0  0  1  1  1    1.0       1     0.0
4   1  1  1  0  0  0  1  0  0  1  1  0  1  0  0  1    1.0       1     0.0
5   1  1  1  0  0  0  1  0  0  1  0  0  1  1  0  0    1.0       1     0.0
6   1  1  0  0  0  1  0  0  1  1  0  0  0  0  1  1    1.0       1     0.0
7   1  1  1  0  0  0  1  0  1  1  0  0  0  0  0  1    1.0       1     0.0
8   0  1  0  1  0  0  1  0  1  0  1  0  1  1  1  0    2.0       1     0.0
9   1  1  0  1  1  0  1  0  0  0  0  0  1  1  0  0    2.0       1     0.0
10  1  0  0  1  0  1  0  0  0  0  1  0  1  1  1  0    2.0       1     0.0
11  1  1  0  1  1  0  1  0  0  0  1  0  1  0  0  1    2.0       1     0.0
12  1  0  1  1  0  1  0  0  1  0  1  0

([[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1],
  [1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
  [1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0],
  [0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1],
  [1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1],
  [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1],
  [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1],
  [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1],
  [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
  [1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1],
  [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1],
  [1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1],
  [1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1],
  [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1],
  [1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1],
  [1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
  [1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1],
  [1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1],
  [0, 0, 0, 

In [17]:
find_valid_y_3(df, num_reads=100, max_chain_strength=50, chain_break_fraction=False)

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
1   1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
2   1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
3   1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
4   1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
5   1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
6   1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
8   1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
9   1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
10  1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
11  1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
12  1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
25  1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
28  1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
30  1  1  0  1  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
35  1  0  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   1  1  1  0  0  0  1  0  1  1  0  0  1  1  0  0    2.0      59
1   1  1  1  0  0  0  1  0  1  1  0  0  1  1  0  0    2.0      10
2   1  1  1  0  0  0  1  0  1  1  0  0  1  1  0  0    2.0       2
3   1  1  1  0  0  0  1  0  1  1  0  0  1  1  0  0    2.0       5
4   1  1  1  0  0  0  1  0  1  1  0  0  1  1  0  0    2.0       1
5   1  1  1  0  0  0  1  0  1  1  0  0  1  1  0  0    2.0       1
8   0  0  0  1  1  1  0  1  0  0  0  1  0  0  1  1    3.0       1
6   0  0  0  1  1  1  0  1  0  0  1  1  0  0  1  1    5.0      15
7   0  0  0  1  1  1  0  1  0  0  1  1  0  0  1  1    5.0       4
9   0  0  0  1  1  1  0  1  0  0  1  1  0  0  1  1    5.0       1
10  0  0  1  1  1  1  0  1  0  0  1  1  0  0  1  1   14.0       1
['BINARY', 11 rows, 100 samples, 16 variables]
    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
1   1  1  0  1  0  1  1  0  0  1  0  1  0  0  0  1    2.0       3
2   1  1  0  1  0  0  1  0  1

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
9   0  0  1  0  0  0  1  1  1  0  1  0  0  1  1  1    0.0       1
11  0  0  1  0  0  1  1  1  1  0  0  0  0  1  1  1    1.0       1
12  1  0  1  0  0  0  0  1  1  0  0  0  1  1  1  1    1.0       1
18  1  0  1  0  0  0  1  1  0  1  0  0  1  0  1  1    1.0       1
19  0  0  1  1  1  0  1  1  0  0  0  0  0  1  1  1    1.0       1
20  1  0  1  0  0  0  0  1  0  0  0  1  1  1  1  1    1.0       1
21  0  0  1  0  1  0  0  1  0  0  1  0  1  1  1  1    1.0       1
24  0  0  1  0  0  1  1  1  1  1  0  0  0  0  1  1    2.0       1
27  0  1  0  1  1  0  0  0  1  0  0  0  1  1  1  1    2.0       1
32  1  0  1  0  0  0  0  1  1  0  0  1  0  0  1  1    2.0       1
38  0  1  0  1  1  1  1  0  0  0  1  0  0  1  0  0    2.0       1
41  1  1  0  1  1  0  0  0  0  0  1  0  1  1  0  0    2.0       1
37  0  1  0  1  1  1  0  0  0  1  0  0  0  1  0  1    3.0       1
3   1  0  1  0  0  1  1  1  1  0  0  0  0  0  1  1    4.0       3
4   1  0  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   0  1  0  1  0  1  1  1  0  0  0  0  0  1  1  1    0.0       1
1   1  1  0  1  0  1  1  0  0  0  1  0  0  1  0  1    0.0       1
2   1  1  0  0  0  0  1  0  0  1  0  1  1  1  0  1    0.0       1
3   0  1  1  1  0  0  1  1  0  0  0  0  0  1  1  1    0.0       3
4   1  0  0  0  0  0  1  0  1  1  1  0  1  1  0  1    0.0       2
5   0  1  1  0  0  0  1  1  1  0  0  0  0  1  1  1    0.0       1
6   0  1  0  1  0  0  1  0  0  0  1  0  1  1  1  1    0.0       1
7   1  1  0  0  1  0  0  0  1  0  1  0  0  1  1  1    0.0       1
8   0  1  0  0  0  1  1  1  1  0  0  0  0  1  1  0    1.0       1
9   1  1  1  0  0  0  0  1  1  0  0  1  0  1  0  1    1.0       1
10  0  1  1  1  0  0  1  1  1  0  0  0  0  1  0  1    1.0       1
11  0  1  0  0  0  1  1  1  1  0  0  0  1  1  0  1    1.0       1
12  0  1  0  1  0  0  1  1  1  0  0  0  1  1  0  1    1.0       1
13  1  1  1  1  0  0  1  1  0  0  0  0  0  1  0  0    1.0       1
14  1  1  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   1  1  0  1  0  0  0  0  0  1  0  1  0  1  1  1    0.0       1
1   0  1  1  1  0  0  1  0  0  0  1  0  0  1  1  1    0.0       1
2   0  1  1  0  0  0  1  1  0  0  0  1  0  1  1  1    0.0       1
3   1  1  0  0  0  0  1  0  0  1  0  1  1  1  0  1    0.0       1
4   1  1  0  0  1  0  1  0  0  1  1  0  0  0  1  1    0.0       1
5   0  1  0  1  0  1  1  0  0  0  1  0  0  1  1  0    1.0       1
6   1  0  1  0  0  0  1  1  1  0  1  0  0  1  1  0    1.0       1
7   1  1  0  1  0  0  1  1  1  1  0  0  0  0  0  1    1.0       1
8   1  1  0  1  0  0  1  0  0  1  1  1  0  1  0  0    1.0       1
9   1  0  0  1  0  0  0  1  0  0  0  1  1  1  0  1    1.0       1
10  0  0  0  1  0  0  1  0  1  1  0  1  0  1  0  1    1.0       1
11  0  0  1  1  0  0  1  0  1  0  1  0  0  1  1  1    1.0       1
12  1  0  0  1  0  0  0  0  0  1  0  1  1  1  0  1    1.0       1
13  1  1  0  0  1  0  1  0  0  1  1  1  0  1  0  0    1.0       1
14  1  1  

...

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   0  0  1  1  0  1  1  1  0  0  0  0  0  1  1  1    1.0       1
1   0  0  1  0  0  0  1  1  0  1  0  0  1  1  1  0    1.0       1
2   1  1  1  0  0  0  0  0  1  0  0  0  1  1  1  1    1.0       1
3   1  0  1  0  0  0  0  0  1  1  0  1  0  1  1  1    1.0       1
4   1  0  1  1  0  1  0  0  0  1  0  0  0  1  0  1    1.0       1
5   0  1  1  1  0  0  1  0  0  0  1  0  1  1  0  1    1.0       1
6   1  0  0  0  0  0  0  0  1  1  1  1  0  1  1  0    1.0       1
7   1  0  0  1  0  0  1  0  0  1  1  1  0  1  1  0    1.0       1
8   1  0  1  0  0  0  0  0  0  1  0  1  1  1  0  1    1.0       1
9   1  0  1  0  0  0  1  0  1  1  1  0  0  1  0  0    1.0       1
10  1  0  0  0  0  0  1  1  1  1  1  0  1  1  0  0    1.0       1
11  1  0  0  1  1  0  0  0  0  1  1  1  0  1  0  1    1.0       1
12  0  0  0  1  0  1  1  1  0  1  0  0  1  1  1  0    2.0       1
13  1  1  0  1  1  0  1  0  0  0  0  0  0  1  1  1    2.0       1
14  1  1  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   1  1  1  0  0  0  0  0  0  0  1  1  0  1  1  1    0.0       1
1   1  1  0  1  0  0  0  0  0  0  1  1  0  1  1  1    0.0       1
2   1  0  1  1  0  0  0  0  0  1  0  1  0  1  1  1    1.0       1
3   1  1  0  0  0  1  0  0  0  1  0  1  0  0  1  1    1.0       1
4   1  0  0  1  0  1  1  1  0  0  0  1  0  1  0  1    1.0       1
5   1  1  0  1  1  0  0  1  0  0  0  0  0  1  1  0    1.0       1
6   1  0  0  1  0  0  0  1  0  1  0  1  1  1  0  1    1.0       1
7   1  0  1  0  0  0  0  0  0  1  1  1  0  1  1  0    1.0       1
8   1  0  1  0  1  0  1  1  0  1  1  0  0  1  0  0    1.0       1
9   1  0  1  0  0  0  0  0  0  1  1  0  1  0  1  1    1.0       1
10  1  1  0  1  1  0  0  0  0  0  1  0  0  0  1  1    1.0       1
11  0  0  0  0  0  0  1  1  0  0  1  1  1  0  1  1    1.0       1
98  0  0  1  1  0  1  1  1  0  1  0  0  0  1  0  1    1.0       1
12  1  0  1  0  0  0  0  0  1  1  0  1  0  0  1  1    2.0       1
13  1  1  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   0  0  0  0  0  1  1  1  0  0  1  0  1  1  1  1    0.0       1
1   1  0  0  0  1  1  1  0  0  1  1  0  0  1  0  1    0.0       1
2   1  0  0  1  0  0  0  0  1  0  1  0  1  1  1  1    1.0       1
3   1  1  0  0  1  1  0  0  1  0  0  0  0  1  1  1    1.0       1
4   0  1  1  0  0  0  1  0  1  0  0  0  1  1  1  1    1.0       1
5   1  0  0  0  1  1  0  1  1  0  0  0  0  1  1  1    1.0       1
6   1  0  1  0  0  0  1  1  1  0  1  0  0  0  1  1    1.0       1
7   0  0  1  0  0  1  0  0  1  1  1  0  0  1  1  1    1.0       1
8   1  0  0  0  1  1  0  0  1  0  1  0  0  1  0  1    1.0       1
9   0  1  0  0  0  1  1  0  1  0  1  0  1  1  0  1    1.0       1
10  1  0  1  0  1  1  0  0  0  0  1  0  0  1  1  1    1.0       1
11  1  1  0  1  1  0  1  0  0  0  1  0  0  0  1  1    1.0       1
12  0  0  0  0  1  1  1  0  0  0  1  0  1  1  0  1    1.0       1
13  0  0  1  0  1  0  1  0  0  1  0  0  1  1  1  1    1.0       1
14  0  0  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   1  1  0  0  0  0  1  0  1  0  1  0  1  1  0  1    0.0       1
1   1  0  0  1  0  0  0  1  1  0  0  1  0  1  1  1    1.0       1
2   0  1  0  0  0  1  0  0  0  0  0  1  1  1  1  1    1.0       1
3   1  0  0  1  1  0  1  0  0  0  1  0  1  1  0  1    1.0       1
4   0  0  1  0  1  0  1  0  0  0  1  1  0  1  0  1    1.0       1
5   0  0  1  0  1  0  0  0  0  1  0  1  0  1  1  1    1.0       1
6   1  1  0  0  1  1  1  0  0  0  0  1  0  1  0  1    1.0       1
7   1  1  0  0  0  0  1  0  1  1  0  1  0  0  1  1    1.0       1
8   1  0  0  1  1  0  0  0  0  1  1  0  0  1  1  0    1.0       1
9   1  0  0  0  0  0  1  1  0  1  0  1  1  0  1  1    1.0       1
10  1  1  0  0  1  0  1  0  1  1  0  0  0  1  1  0    1.0       1
11  1  0  0  1  0  0  0  0  0  1  1  1  0  1  1  0    1.0       1
12  0  1  1  1  0  0  1  0  0  1  0  0  0  1  1  0    1.0       1
13  1  1  0  0  0  0  1  0  1  1  1  0  0  1  1  1    1.0       1
14  1  0  

([[1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
  [0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
  [1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1],
  [1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1],
  [0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1],
  [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
  [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1],
  [1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
  [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1],
  [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1],
  [0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1],
  [1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1],
  [0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
  [1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1],
  [0, 1, 1, 

In [18]:
find_valid_y_3(df, num_reads=200, max_chain_strength=50, chain_break_fraction=True)

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
37   0  1  0  0  0  1  1  1  0  0  0  1  0  1  1  1    0.0       1  0.9375
42   0  1  0  0  0  1  1  1  0  0  0  1  0  1  1  1    0.0       1     1.0
121  0  1  0  0  0  1  1  1  0  0  0  1  0  1  1  1    0.0       1     1.0
25   1  0  0  0  1  1  0  1  1  0  1  0  0  1  0  1    1.0       2  0.9375
26   1  0  0  0  1  1  0  1  1  0  0  0  0  1  0  1    1.0       1     1.0
27   1  0  0  0  1  1  0  1  1  0  1  0  0  1  0  1    1.0       2     1.0
30   0  1  0  0  0  1  1  1  0  0  0  1  0  0  1  1    1.0       1     1.0
31   1  0  0  0  1  1  0  1  1  0  1  0  0  1  0  1    1.0       1     1.0
35   1  0  0  0  1  1  0  1  1  0  0  0  0  1  0  1    1.0       4   0.875
38   0  1  0  0  0  1  1  1  0  0  0  1  0  0  1  1    1.0       1  0.9375
39   0  1  0  0  0  1  1  1  0  0  1  1  0  0  1  1    1.0       2     1.0
41   0  1  0  0  0  1  1  1  0  0  0  1  0  0  1  1    1.0       2     1.0
44   0  1  1  0  0  1  0 

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
17  0  0  1  0  1  1  0  1  0  1  1  0  1  1  1  0   11.0       2  0.8125
18  0  0  1  0  1  1  0  1  0  1  1  0  1  1  1  0   11.0       1  0.8125
19  0  0  1  0  1  1  0  1  0  1  1  0  1  1  1  0   11.0       1  0.8125
1   1  1  0  1  1  0  1  0  1  0  0  1  1  1  0  1   17.0      39   0.875
2   1  1  0  1  1  0  1  0  1  0  0  1  1  1  0  1   17.0      34   0.875
3   1  1  0  1  1  0  1  0  1  0  0  1  1  1  0  1   17.0      13   0.875
5   1  1  0  1  1  0  1  0  1  0  0  1  1  1  0  1   17.0       9   0.875
7   1  1  0  1  1  0  1  0  1  0  0  1  1  1  0  1   17.0       4  0.8125
10  1  1  0  1  1  0  1  0  1  0  0  1  1  1  0  1   17.0       2  0.8125
12  1  1  0  1  1  0  1  0  1  0  0  1  1  1  0  1   17.0       1   0.875
13  1  1  0  1  1  0  1  0  1  0  0  1  1  1  0  1   17.0       1   0.875
15  1  1  0  1  1  0  1  0  1  0  0  1  1  1  0  1   17.0       1   0.875
14  1  1  0  1  1  0  1  0  1  0  0  1

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
8   1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       6    0.25
15  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       6  0.3125
17  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       1  0.1875
18  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       1    0.25
19  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       1  0.1875
21  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       2  0.3125
23  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       2    0.25
24  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       1   0.375
26  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       1    0.25
27  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       2   0.375
32  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       2  0.1875
34  1  1  0  1  0  0  1  1  1  1  0  0  0  1  0  0    1.0       2    0.25
35  1  1  0  1  0  0  1  1  1  1  0  0

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
2    0  1  0  0  0  0  1  1  1  0  0  0  1  1  1  1    0.0       1     0.0
3    0  1  0  1  0  0  1  0  0  0  1  1  0  1  1  1    0.0       1     0.0
4    0  0  0  1  0  0  1  1  0  0  1  0  1  1  1  1    0.0       2     0.0
5    0  1  0  0  0  0  1  0  0  0  1  1  1  1  1  1    0.0       2     0.0
6    0  0  0  0  1  0  1  0  0  1  1  1  0  1  1  1    0.0       1     0.0
7    0  0  0  1  0  0  1  0  0  1  1  1  0  1  1  1    0.0       1     0.0
8    0  0  0  0  0  0  1  1  0  0  1  1  1  1  1  1    0.0       1     0.0
9    0  0  0  1  0  0  1  1  0  0  1  1  0  1  1  1    0.0       3     0.0
10   0  0  1  1  0  0  1  1  0  0  1  0  0  1  1  1    0.0       1     0.0
11   1  0  0  1  0  0  1  1  1  0  1  0  0  1  0  1    0.0       1     0.0
12   1  0  0  1  0  0  0  1  0  0  1  1  0  1  1  1    0.0       1     0.0
61   0  0  0  1  0  0  1  1  0  0  1  1  0  1  1  1    0.0       3  0.0625
63   0  1  0  1  0  0  1 

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0    1  1  1  0  0  0  0  0  1  1  0  0  0  1  1  1    0.0       1     0.0
1    1  1  1  0  0  1  0  1  0  0  0  0  0  1  1  1    0.0       1     0.0
2    1  0  0  0  0  0  0  1  1  1  0  0  1  1  1  1    0.0       1     0.0
3    1  1  0  0  0  1  0  1  1  0  0  0  0  1  1  1    0.0       1     0.0
4    1  1  0  0  0  1  1  1  0  0  0  0  1  1  0  1    0.0       2     0.0
5    0  1  0  1  0  0  1  1  1  0  0  0  0  1  1  1    0.0       2     0.0
6    1  0  0  1  0  0  1  1  0  0  1  0  1  1  0  1    0.0       1     0.0
7    1  0  1  0  0  0  0  1  0  0  1  1  0  1  1  1    0.0       1     0.0
8    1  0  0  0  1  0  1  1  1  1  0  0  0  1  0  1    0.0       1     0.0
9    1  1  1  0  0  0  1  1  1  0  0  0  0  1  0  1    0.0       1     0.0
10   1  1  0  0  1  0  1  0  1  0  1  0  0  1  0  1    0.0       1     0.0
11   0  1  0  1  1  0  1  0  0  0  1  0  0  1  1  1    0.0       1     0.0
12   1  0  0  0  0  0  1 

...

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0    1  0  1  0  0  0  0  1  0  0  1  0  1  1  1  1    0.0       1     0.0
1    0  1  1  0  0  1  1  0  0  1  0  0  1  1  0  1    1.0       1     0.0
2    0  0  0  0  0  1  1  0  1  1  0  0  1  1  1  1    1.0       1     0.0
3    0  0  0  1  0  0  1  1  0  0  1  0  1  1  1  0    1.0       1     0.0
4    0  1  0  1  1  0  1  1  1  0  0  0  0  1  0  1    1.0       1     0.0
5    0  1  0  0  1  0  1  1  0  0  1  0  1  1  1  0    1.0       1     0.0
6    1  0  1  0  0  0  0  1  1  0  0  0  1  1  0  1    1.0       1     0.0
7    1  1  0  0  0  1  1  1  0  0  0  1  0  0  1  1    1.0       1     0.0
8    1  1  0  0  0  0  1  1  1  0  0  0  1  1  0  0    1.0       1     0.0
9    1  1  0  0  0  0  1  0  0  0  1  0  1  1  1  1    1.0       1     0.0
10   1  1  0  1  0  0  1  0  0  1  1  0  0  1  0  1    1.0       1     0.0
11   0  1  1  0  0  0  1  1  0  1  0  1  0  0  1  0    2.0       1     0.0
12   0  0  1  1  0  0  1 

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0    1  1  1  0  0  1  0  1  0  0  0  0  0  1  1  1    0.0       1     0.0
1    1  1  1  0  0  0  1  0  0  1  0  0  1  0  1  1    1.0       1     0.0
2    0  1  1  0  0  0  1  0  1  0  0  0  1  1  1  1    1.0       1     0.0
3    1  0  0  1  0  0  1  1  1  0  0  1  0  1  0  1    1.0       1     0.0
4    0  1  0  1  0  0  1  1  0  0  0  0  1  0  1  1    1.0       1     0.0
5    1  1  0  0  0  1  1  1  0  0  0  1  0  0  0  1    1.0       1     0.0
6    1  1  0  1  0  0  1  0  1  1  0  0  0  1  1  0    1.0       1     0.0
7    1  1  0  0  0  1  1  1  1  1  0  0  0  1  0  0    1.0       1     0.0
8    1  1  1  1  0  0  1  1  0  1  0  0  0  1  0  0    1.0       1     0.0
9    0  1  0  0  0  1  1  1  1  0  0  0  0  1  1  0    1.0       1     0.0
10   1  1  1  0  0  1  1  1  0  0  1  0  0  0  0  1    1.0       1     0.0
11   0  1  1  0  1  1  1  0  0  0  0  0  0  1  0  1    1.0       1     0.0
12   1  0  0  1  0  0  1 

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0    0  0  0  0  0  0  1  0  1  1  1  0  1  1  1  1    0.0       1     0.0
1    0  1  0  0  0  0  1  0  0  1  0  1  1  1  1  1    0.0       1     0.0
2    1  1  0  0  1  0  0  0  0  1  0  1  0  1  1  1    0.0       1     0.0
3    1  0  0  0  0  0  0  0  1  1  1  0  1  1  1  1    0.0       1     0.0
4    1  1  1  1  0  0  1  1  0  0  0  0  0  0  0  1    1.0       1     0.0
5    1  0  1  0  0  0  1  1  0  1  0  0  0  1  1  1    1.0       1     0.0
6    1  0  1  1  0  0  1  0  1  1  0  0  0  1  0  1    1.0       1     0.0
7    1  0  1  0  0  1  1  1  0  1  0  0  0  0  0  1    1.0       1     0.0
8    1  0  0  0  0  0  1  0  1  1  0  1  1  1  0  1    1.0       1     0.0
9    0  0  1  0  0  1  1  1  0  1  0  0  1  1  0  1    1.0       1     0.0
10   0  0  0  0  0  1  0  0  1  0  1  1  0  1  1  1    1.0       1     0.0
11   0  0  0  0  1  0  1  0  0  1  1  0  1  1  1  0    1.0       1     0.0
12   1  1  1  0  0  0  0 

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0    1  1  0  0  1  0  0  1  1  0  0  0  0  1  1  1    0.0       1     0.0
1    1  1  0  0  0  0  1  1  0  1  0  0  1  1  1  0    0.0       1     0.0
2    1  1  0  0  0  0  1  1  0  1  0  1  0  1  1  0    0.0       1     0.0
3    1  1  1  0  1  1  1  0  0  0  0  0  0  1  0  1    1.0       1     0.0
4    0  1  1  0  1  1  1  0  0  0  0  0  0  1  0  1    1.0       1     0.0
5    0  0  1  0  1  0  1  1  1  0  0  0  0  1  1  1    1.0       1     0.0
6    0  1  0  0  1  0  1  0  0  1  1  1  0  1  1  0    1.0       1     0.0
7    1  1  0  0  1  1  0  1  0  0  0  1  0  1  0  1    1.0       1     0.0
8    0  0  1  1  1  0  0  0  0  1  1  0  0  1  1  1    1.0       1     0.0
9    1  0  0  0  0  0  1  1  1  0  1  0  1  1  1  0    1.0       1     0.0
10   1  1  1  1  0  0  1  1  0  0  0  0  0  1  0  0    1.0       1     0.0
11   1  1  0  1  0  0  1  0  0  0  1  1  0  1  1  0    1.0       1     0.0
12   0  1  0  0  0  0  1 

([[0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1],
  [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1],
  [0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
  [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
  [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1],
  [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1],
  [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1],
  [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1],
  [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
  [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1],
  [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1],
  [0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1],
  [1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
  [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1],
  [1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1],
  [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1],
  [0, 1, 0, 

In [19]:
find_valid_y_3(df, num_reads=200, max_chain_strength=50, chain_break_fraction=False)

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
1    1  1  0  1  0  0  0  0  0  0  1  0  1  1  1  1    0.0       1
26   1  1  0  1  1  0  0  0  0  0  1  0  0  1  1  1    0.0       2
77   0  1  0  1  0  0  1  0  0  0  1  1  0  1  1  1    0.0       1
87   1  1  0  1  1  0  0  0  0  0  1  0  0  1  1  1    0.0       1
13   1  1  0  0  0  0  1  0  0  0  1  1  1  0  1  1    1.0       1
28   0  1  0  0  0  0  1  0  0  0  1  1  1  0  1  1    1.0       1
34   1  1  0  1  0  0  0  0  1  0  1  0  1  1  0  1    1.0       1
45   0  1  0  0  0  0  1  0  0  0  1  1  1  0  1  1    1.0       1
53   1  1  0  1  1  0  0  0  1  0  1  0  0  1  0  1    1.0       1
55   1  1  0  1  0  0  0  0  1  0  1  0  1  1  0  1    1.0       1
59   1  0  0  1  1  1  0  1  0  0  0  0  0  1  0  1    1.0       1
63   1  1  1  0  0  0  1  0  0  0  1  1  0  0  1  1    1.0       2
65   1  1  1  0  0  0  1  0  0  0  1  1  0  0  1  1    1.0       1
73   1  1  0  1  1  0  0  0  0  0  1  0  1  1  0  1    1.0    

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
12  0  1  0  0  0  0  1  1  1  1  0  0  1  1  1  0    1.0      17
13  0  1  0  0  0  0  1  1  1  1  0  0  1  0  1  1    1.0       9
15  0  1  0  0  0  0  1  1  1  1  0  0  1  1  1  0    1.0       2
22  0  1  0  0  0  0  1  1  1  1  0  0  1  1  1  0    1.0       2
3   1  0  1  0  1  1  0  0  0  0  1  1  0  1  0  1    2.0       1
4   1  0  1  0  1  1  0  0  0  0  1  1  0  1  0  1    2.0       5
5   1  0  1  0  1  1  0  0  0  0  1  1  0  1  0  1    2.0       6
6   1  0  1  0  1  1  0  0  0  0  1  1  0  1  0  1    2.0       5
8   0  1  0  0  0  0  1  1  1  1  0  0  1  1  1  1    2.0       6
9   0  1  0  0  0  0  1  1  1  1  0  0  1  1  1  1    2.0       5
10  0  1  0  0  0  0  1  1  1  1  0  0  1  1  1  1    2.0       9
11  0  1  0  0  0  0  1  1  1  1  0  0  1  1  1  1    2.0      17
19  1  0  1  0  1  1  0  0  0  0  1  1  0  1  0  1    2.0       1
20  0  1  0  0  0  0  1  1  1  1  0  0  1  1  1  1    2.0       1
21  0  1  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
59  1  1  0  0  0  1  1  0  0  1  0  1  0  1  0  1    0.0       1
7   1  1  0  0  0  1  1  0  0  1  0  1  0  0  0  1    1.0       3
16  1  1  0  0  0  1  1  0  0  1  0  1  0  0  0  1    1.0       1
49  1  1  0  0  0  1  1  0  0  1  0  1  0  0  0  1    1.0       1
51  1  1  0  0  0  1  1  0  0  1  0  1  0  0  0  1    1.0       1
60  1  1  0  1  0  1  1  0  0  0  0  1  0  1  0  1    1.0       1
0   1  1  0  1  0  1  1  0  0  1  0  1  0  0  0  1    2.0      22
1   1  1  0  0  1  1  1  0  0  1  0  1  0  0  0  1    2.0      11
2   0  0  1  0  0  0  0  1  1  0  1  0  1  1  1  0    2.0      26
3   1  1  0  1  0  1  1  0  0  1  0  1  0  0  0  1    2.0      10
4   1  1  0  0  1  1  1  0  0  1  0  1  0  0  0  1    2.0       6
5   1  1  0  0  1  1  1  0  0  1  0  1  0  0  0  1    2.0       2
6   1  1  0  1  0  1  1  0  0  1  0  1  0  0  0  1    2.0       2
8   1  1  0  1  0  1  1  0  0  0  0  1  0  0  0  1    2.0       6
9   1  1  

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0    1  0  1  0  0  0  0  1  0  1  0  0  1  1  1  1    0.0       2
1    1  0  0  0  0  0  1  1  0  1  0  1  1  1  0  1    0.0       1
2    1  0  1  0  0  0  1  1  0  1  0  1  0  1  0  1    0.0       1
3    1  0  1  0  0  0  0  1  0  1  0  1  0  1  1  1    0.0       1
4    1  0  0  0  0  1  0  1  0  1  0  0  1  1  1  1    0.0       1
5    0  0  0  1  0  0  1  0  1  1  1  0  0  1  1  1    0.0       1
6    1  1  0  1  1  0  1  0  0  0  1  0  0  1  0  1    0.0       1
129  1  0  1  0  0  0  0  1  0  1  0  0  1  1  1  1    0.0       1
8    1  0  1  0  0  0  0  1  1  1  0  0  1  1  0  1    1.0       1
9    1  0  1  0  0  0  0  0  1  1  1  0  0  0  1  1    1.0       1
10   1  1  0  1  0  0  1  0  1  1  0  0  0  0  0  1    1.0       1
11   1  0  1  0  0  0  1  1  0  1  0  0  1  0  1  1    1.0       3
12   1  0  1  0  0  0  1  1  1  1  0  0  0  1  0  0    1.0       1
13   1  0  1  0  0  0  1  1  0  1  0  0  1  0  0  1    1.0    

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0    0  0  1  0  1  0  1  1  0  0  1  0  0  1  1  1    0.0       1
1    1  0  1  0  1  0  0  1  0  0  1  0  0  1  1  1    0.0       1
2    0  0  0  0  1  0  1  1  0  0  1  0  1  1  1  1    0.0       1
3    0  1  0  0  1  0  1  1  0  0  0  0  1  1  1  1    0.0       1
4    1  1  0  0  0  0  0  0  1  0  1  0  1  1  1  1    0.0       1
5    1  0  0  0  1  0  0  1  0  0  1  1  0  1  1  1    0.0       1
6    1  1  0  0  1  0  0  0  0  0  1  1  0  1  1  1    0.0       1
7    0  0  0  0  0  0  1  1  0  1  0  1  1  1  1  1    0.0       1
122  0  1  0  0  1  0  1  1  0  0  0  1  0  1  1  1    0.0       1
8    1  1  1  1  1  0  0  0  0  0  1  0  0  1  0  1    1.0       1
9    0  0  1  0  1  0  1  1  0  0  1  0  1  1  0  1    1.0       1
10   1  0  0  1  0  1  1  0  1  1  0  0  0  1  0  1    1.0       1
11   1  0  0  1  0  1  0  1  1  1  0  0  0  1  0  1    1.0       1
12   1  0  1  1  0  1  0  0  0  1  0  0  0  1  0  1    1.0    

...

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0    0  0  0  1  0  0  1  0  0  1  1  0  1  1  1  1    0.0       1
1    1  1  1  0  0  0  0  0  0  1  0  1  0  1  1  1    0.0       1
2    0  1  1  0  1  0  1  1  0  0  0  0  0  1  1  1    0.0       1
3    0  0  0  0  1  0  1  1  0  1  0  0  1  1  1  1    0.0       1
4    0  1  0  0  1  0  1  1  0  0  0  1  0  1  1  1    0.0       1
5    1  1  0  1  1  0  1  1  0  1  0  0  0  0  0  1    1.0       1
6    0  1  1  1  0  0  1  0  0  0  0  1  0  1  0  1    1.0       1
7    0  1  1  0  0  0  1  0  0  1  0  0  1  0  1  1    1.0       1
8    1  1  0  0  0  0  1  1  1  1  0  0  1  0  0  1    1.0       1
9    0  1  0  1  0  1  1  0  0  0  0  0  1  1  1  1    1.0       1
10   0  1  1  1  0  1  1  0  0  0  0  0  0  1  1  1    1.0       1
11   1  0  1  1  0  0  0  0  0  1  0  0  1  1  1  1    1.0       1
12   0  1  1  0  0  0  1  1  0  1  0  0  1  0  1  1    1.0       1
13   1  0  0  1  0  0  1  1  0  1  1  0  0  1  0  1    1.0    

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0    1  0  0  0  1  0  0  1  0  0  1  1  0  1  1  1    0.0       1
1    0  0  0  1  1  0  1  1  0  0  1  0  0  1  1  1    0.0       1
2    1  0  0  0  1  0  1  1  0  1  1  0  1  1  0  0    1.0       1
3    1  0  1  0  0  0  1  1  0  0  1  0  1  0  0  1    1.0       1
4    1  1  0  0  0  0  1  0  0  0  1  1  1  1  0  0    1.0       1
5    1  1  0  0  0  0  0  1  0  0  1  1  1  0  1  1    1.0       1
6    1  1  0  0  0  0  1  1  1  0  0  0  1  0  1  1    1.0       1
7    1  0  1  0  0  0  0  0  1  1  0  1  0  1  1  1    1.0       1
8    1  0  0  0  0  0  1  1  1  0  1  1  0  1  1  0    1.0       1
9    1  0  0  0  1  0  0  1  1  1  1  0  0  1  1  0    1.0       1
10   1  0  0  1  1  0  0  1  1  0  1  0  0  1  0  1    1.0       1
11   1  0  0  1  1  0  0  1  0  0  1  0  0  1  1  0    1.0       1
12   1  1  0  0  0  1  1  0  0  0  1  1  0  1  0  0    1.0       1
13   1  1  0  0  1  0  1  0  0  0  1  1  0  1  0  0    1.0    

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0    1  0  0  0  0  0  1  1  0  1  1  1  0  0  1  1    0.0       1
1    1  0  0  0  0  1  0  1  1  0  0  0  1  1  1  1    1.0       1
2    0  0  1  0  0  1  0  1  0  0  0  1  0  1  1  1    1.0       1
3    0  0  0  1  0  1  1  1  1  0  0  0  0  1  0  1    1.0       1
4    0  0  0  0  1  1  0  1  0  0  0  0  1  1  1  1    1.0       1
5    0  0  0  0  1  1  1  1  1  0  0  0  0  1  0  1    1.0       1
6    0  0  0  0  1  0  0  1  1  0  0  0  1  1  1  1    1.0       1
7    0  0  0  1  1  0  0  1  1  0  0  0  0  1  1  1    1.0       1
8    1  0  0  1  0  0  1  1  0  1  0  0  1  0  0  1    1.0       1
9    1  0  0  0  0  0  1  1  0  1  0  0  1  1  1  1    1.0       1
10   0  0  0  0  1  0  1  1  1  1  0  0  0  1  1  0    1.0       1
11   1  0  0  0  0  1  1  1  0  0  1  0  1  1  0  0    1.0       1
12   0  0  0  0  0  1  1  1  0  0  1  1  0  1  1  0    1.0       1
13   1  0  0  1  0  0  0  1  0  1  1  1  0  1  1  0    1.0    

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0    1  0  0  0  0  0  0  1  0  0  1  1  1  0  1  1    1.0       1
1    1  1  0  1  0  0  1  1  0  0  0  1  0  0  1  1    1.0       1
2    1  0  0  0  0  1  1  1  0  0  1  0  1  1  0  0    1.0       1
3    1  1  0  0  1  0  0  1  0  0  1  0  1  1  1  0    1.0       1
4    1  0  0  0  1  0  1  1  1  0  1  0  0  0  1  1    1.0       1
5    1  0  1  0  0  0  0  1  0  0  1  1  1  1  0  1    1.0       1
6    1  0  0  0  0  0  0  0  0  1  1  1  1  1  1  0    1.0       1
7    0  0  0  1  0  1  0  0  0  1  1  0  1  1  1  1    1.0       1
8    0  0  1  0  0  1  0  0  1  0  1  0  0  1  1  1    1.0       1
9    0  0  1  1  1  0  0  1  0  0  1  0  0  1  1  1    1.0       1
10   0  0  1  0  1  0  0  1  0  0  1  1  0  1  1  1    1.0       1
11   1  0  0  0  1  0  0  0  0  1  1  0  1  0  1  1    1.0       1
12   0  0  0  0  0  1  0  1  1  1  0  1  0  1  1  1    1.0       1
13   1  0  0  1  1  0  0  1  0  0  1  0  1  1  0  1    1.0    

([[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1],
  [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
  [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1],
  [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
  [1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1],
  [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1],
  [1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1],
  [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
  [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1],
  [1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1],
  [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1],
  [1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1],
  [0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1],
  [1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1],
  [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1],
  [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
  [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1],
  [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1],
  [1, 1, 0, 

In [20]:
find_valid_y_3(df, num_reads=20, max_chain_strength=50, chain_break_fraction=True)

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
9   1  1  1  0  0  0  0  0  0  1  1  1  0  0  1  1    1.0       1     1.0
13  1  1  1  0  1  0  0  0  0  1  1  0  0  0  1  1    1.0       1  0.9375
1   0  1  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0       1  0.9375
2   0  1  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0       2  0.9375
3   0  1  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0       1     1.0
8   0  1  0  0  0  1  0  0  0  1  1  0  1  1  1  1    2.0       1     1.0
15  1  0  1  0  1  0  0  0  0  1  1  1  0  1  1  1    3.0       1     1.0
14  0  1  0  1  0  0  1  0  1  1  1  1  0  1  0  1    5.0       1     1.0
16  1  0  1  0  1  1  0  1  0  0  0  0  1  1  1  0    5.0       1     1.0
0   0  1  0  0  0  1  0  0  0  1  1  0  1  1  0  1    6.0       1     1.0
4   0  1  0  0  0  1  0  0  0  1  1  0  1  1  0  1    6.0       1     1.0
5   0  1  0  0  0  1  0  0  0  1  1  0  1  1  0  1    6.0       2  0.9375
6   0  1  0  0  0  0  0  0  0  1  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
1   0  0  1  0  0  0  1  1  0  1  0  1  0  1  1  1    0.0       1     0.0
0   1  1  0  1  0  0  1  0  0  0  1  1  0  1  0  0    1.0       1  0.0625
2   1  1  0  1  1  0  1  0  0  0  1  0  0  1  0  0    1.0       1   0.125
3   0  0  0  0  0  1  0  1  0  1  0  1  1  1  1  1    1.0       1     0.0
4   1  0  0  1  1  0  1  0  0  0  1  1  0  1  0  1    1.0       1     0.0
5   0  0  0  0  0  0  1  1  0  1  0  1  1  0  1  1    1.0       1     0.0
6   0  0  1  0  0  0  1  1  0  1  0  0  1  0  1  1    1.0       1   0.125
7   0  0  0  0  0  1  0  1  1  1  0  0  1  0  1  1    2.0       1   0.125
8   0  0  0  0  0  1  0  1  0  1  0  1  1  0  1  1    2.0       1     0.0
10  0  0  0  0  0  1  1  1  1  1  0  0  1  0  0  1    2.0       1     0.0
11  0  0  1  0  0  1  1  1  0  1  0  0  1  0  0  1    2.0       1     0.0
15  1  1  0  1  1  0  1  0  0  0  1  1  0  1  0  0    2.0       1  0.1875
19  0  0  0  0  0  1  1  1  0  1  0  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  0  0  0  0  0  1  1  1  1  0  1  0  1  0  1    0.0       1     0.0
1   1  1  0  0  1  0  0  1  1  0  0  0  1  1  0  1    1.0       1     0.0
2   0  0  1  0  0  0  1  1  1  0  1  1  0  1  0  1    1.0       1     0.0
3   1  1  0  0  0  0  0  1  1  1  0  0  1  0  1  1    1.0       1     0.0
4   1  0  0  1  0  1  0  1  1  0  0  0  0  1  1  1    1.0       1     0.0
5   1  0  1  1  0  0  0  0  1  1  0  0  0  1  1  1    1.0       1     0.0
6   0  1  0  0  0  1  1  0  1  0  0  0  1  1  0  1    1.0       1     0.0
7   1  0  1  0  0  0  0  1  1  1  1  0  0  1  1  1    2.0       1     0.0
8   1  0  0  0  1  0  0  1  1  1  0  0  1  1  0  0    2.0       1     0.0
9   1  0  1  1  1  0  0  1  1  0  0  0  0  1  0  1    2.0       1     0.0
10  0  1  1  0  0  0  0  1  1  0  0  1  0  0  1  1    2.0       1     0.0
11  0  0  1  1  0  0  1  0  0  1  1  0  0  1  0  1    2.0       1     0.0
12  1  0  1  0  0  0  1  1  1  0  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  0  0  0  1  0  1  1  0  1  0  0  1  0  0  1    1.0       1     0.0
1   1  0  1  0  0  0  1  1  1  0  1  0  0  0  1  1    1.0       1     0.0
2   0  0  0  0  1  0  1  1  0  1  0  1  1  0  0  1    2.0       1     0.0
3   1  1  0  0  1  0  0  1  0  0  0  1  1  1  0  0    2.0       1     0.0
4   1  1  0  0  1  0  1  0  0  1  0  1  1  0  0  1    2.0       1     0.0
5   0  1  0  1  0  0  1  1  0  1  0  1  0  0  1  0    2.0       1     0.0
6   0  0  0  1  1  0  0  0  0  1  1  0  1  0  1  1    2.0       1     0.0
7   1  1  1  0  0  0  1  1  0  1  0  1  0  0  1  0    2.0       2     0.0
8   1  0  0  1  0  1  0  0  0  1  0  1  1  1  0  1    2.0       1     0.0
9   0  0  0  1  1  1  0  1  0  1  0  0  0  1  0  1    3.0       1     0.0
10  1  1  1  0  0  1  0  0  0  1  1  0  0  0  0  1    3.0       1     0.0
11  1  1  0  1  0  0  1  1  0  1  0  1  0  0  1  1    3.0       1     0.0
12  0  1  0  0  1  0  1  0  0  0  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  1  0  0  1  0  1  0  1  0  0  0  1  1  0  1    1.0       1     0.0
1   0  1  1  0  1  0  1  0  0  1  1  0  0  0  1  1    1.0       1     0.0
2   0  0  0  0  1  0  1  0  0  0  1  1  1  1  0  1    1.0       1     0.0
3   1  1  1  1  0  0  1  1  0  1  0  0  0  0  0  0    2.0       1     0.0
4   1  0  1  0  0  1  1  1  0  1  1  0  0  0  1  0    2.0       1     0.0
5   1  1  1  0  0  1  0  0  0  0  1  1  0  0  1  1    2.0       1     0.0
6   1  0  0  0  0  1  0  0  1  1  0  1  0  0  1  1    2.0       1     0.0
7   0  1  0  1  0  1  0  1  0  0  1  0  0  0  1  1    3.0       1     0.0
8   1  1  0  0  0  1  1  1  0  1  1  0  0  0  0  0    3.0       1     0.0
9   0  1  0  1  0  1  1  0  0  1  1  1  0  0  0  1    4.0       1     0.0
10  0  1  0  1  1  0  0  0  1  1  0  0  0  0  1  0    5.0       1     0.0
11  1  1  0  0  0  0  1  0  1  0  0  1  1  0  0  0    5.0       1     0.0
12  0  1  0  0  1  0  0  1  1  0  1  1

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   0  1  0  1  0  0  1  0  0  0  1  0  1  1  1  1    0.0       1     0.0
1   1  1  0  0  1  0  0  1  1  0  0  0  0  1  1  1    0.0       1     0.0
2   0  0  0  1  0  0  1  1  1  0  1  0  0  0  1  1    1.0       1     0.0
3   1  0  1  0  0  0  1  1  0  0  1  0  0  1  1  1    1.0       1     0.0
4   1  0  0  1  1  0  0  1  0  1  1  0  0  0  1  0    2.0       1     0.0
5   1  1  0  1  1  0  0  0  1  1  0  0  0  1  0  0    2.0       1     0.0
6   1  1  0  0  0  1  1  0  0  1  0  1  1  0  0  1    2.0       1     0.0
7   0  1  0  1  0  0  1  1  1  0  1  0  0  1  1  1    2.0       1     0.0
8   1  1  0  1  1  0  0  0  0  0  0  0  1  1  1  0    2.0       1     0.0
9   1  1  0  0  1  1  1  1  0  0  0  1  0  1  0  1    3.0       1     0.0
10  1  1  0  0  1  0  0  0  0  0  1  0  0  1  1  1    3.0       1     0.0
11  1  1  0  1  1  0  1  0  0  1  1  0  0  1  1  0    3.0       1     0.0
12  0  0  1  1  0  0  1  0  0  0  0  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  0  0  0  0  0  1  1  0  0  1  1  0  1  1  1    1.0       1     0.0
1   0  1  0  1  0  1  1  1  0  0  1  0  0  0  1  1    1.0       1     0.0
2   0  0  0  0  0  0  1  1  0  1  1  1  0  1  1  0    2.0       1     0.0
3   0  0  0  0  0  1  1  1  1  0  1  0  1  0  0  1    2.0       1     0.0
4   0  0  0  1  0  1  1  0  0  1  1  0  1  1  0  0    2.0       1     0.0
5   0  0  0  1  0  1  1  0  0  1  1  0  1  0  1  1    2.0       1     0.0
6   1  1  1  0  1  0  0  0  0  0  0  1  0  1  1  0    2.0       1     0.0
7   1  1  0  1  0  0  0  0  1  1  1  1  0  0  1  0    5.0       1     0.0
8   0  1  0  1  0  1  1  1  0  1  0  1  0  0  0  0    5.0       1     0.0
9   0  0  1  0  0  1  1  0  1  0  0  0  1  1  0  0    5.0       1     0.0
10  0  1  0  0  1  1  1  0  0  0  1  1  0  0  0  0    5.0       1     0.0
11  0  0  0  0  1  1  0  0  1  1  0  1  0  0  1  0    6.0       1     0.0
12  0  0  0  0  0  0  1  1  0  1  1  1

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  1  0  1  0  0  0  0  0  0  1  1  1  1  0  1    1.0       1     0.0
1   0  0  0  1  0  0  1  1  1  0  1  0  0  1  1  0    1.0       1     0.0
2   0  1  1  1  0  0  1  0  1  0  0  0  0  1  1  1    1.0       1     0.0
3   1  0  0  1  1  0  0  0  1  0  1  0  1  1  0  1    2.0       1     0.0
4   1  1  0  1  0  0  1  0  0  0  0  1  1  1  0  0    2.0       1     0.0
5   0  0  0  1  1  0  1  0  0  0  1  0  1  1  1  0    2.0       1     0.0
6   0  0  0  1  1  1  1  0  0  1  1  0  0  1  1  0    2.0       1     0.0
7   1  1  1  1  0  0  1  0  0  1  0  0  0  1  1  1    2.0       1     0.0
8   0  0  0  1  1  0  1  1  0  0  1  0  1  1  1  0    2.0       1     0.0
9   0  0  0  1  0  0  1  1  0  1  0  1  0  1  0  1    2.0       1     0.0
10  0  0  1  0  0  1  1  1  0  0  1  0  0  1  0  0    5.0       1     0.0
11  1  1  0  1  0  0  0  0  0  1  0  0  1  0  0  1    5.0       1     0.0
12  0  0  1  1  0  1  0  1  0  0  1  0

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0   1  0  1  1  0  0  0  1  0  1  1  0  0  0  1  1    1.0       1     0.0
1   1  1  1  0  1  0  0  0  0  0  0  1  0  0  1  1    2.0       1     0.0
2   0  1  1  0  1  0  0  0  0  0  1  1  0  0  1  1    2.0       1     0.0
3   0  0  1  1  0  1  0  1  1  0  0  0  0  1  0  1    2.0       1     0.0
4   0  1  0  0  1  0  1  1  1  0  1  0  0  0  1  0    2.0       1     0.0
5   1  0  1  1  0  1  0  0  0  0  1  0  1  0  0  1    3.0       1     0.0
6   0  0  1  0  0  1  0  1  0  1  1  1  0  0  1  1    4.0       1     0.0
7   1  0  0  1  0  1  0  0  0  0  0  1  1  0  1  1    5.0       1     0.0
8   1  0  1  0  1  1  0  1  0  0  1  1  0  0  0  1    5.0       1     0.0
9   0  0  0  0  0  1  0  0  1  0  1  1  0  1  0  1    5.0       1     0.0
10  1  1  0  0  1  1  0  1  0  0  1  1  0  0  1  0    5.0       1     0.0
11  0  1  0  1  1  1  0  1  0  0  0  0  1  0  1  1    5.0       1     0.0
12  0  1  1  0  0  1  0  0  1  1  1  0

([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1],
  [1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1],
  [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1],
  [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1],
  [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1],
  [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1],
  [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1],
  [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1],
  [1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1],
  [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1],
  [0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1],
  [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1],
  [1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1],
  [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1],
  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
  [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1],
  [1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1],
  [1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1],
  [1, 1, 0, 

In [21]:
find_valid_y_3(df, num_reads=20, max_chain_strength=50, chain_break_fraction=False)

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
6   1  0  0  0  0  1  1  1  1  0  1  0  0  1  0  1    0.0       1
0   1  0  0  0  0  1  1  1  1  1  1  0  0  1  0  1    2.0       1
1   1  0  0  0  0  1  1  1  1  1  1  0  0  1  0  1    2.0       1
5   1  0  0  0  0  1  1  1  1  1  1  0  0  1  0  1    2.0       1
7   0  1  0  0  0  1  1  1  1  0  1  0  0  1  0  1    2.0       1
12  1  0  0  0  1  0  0  1  0  1  1  0  1  1  0  1    2.0       1
13  0  1  0  0  1  1  1  0  1  0  0  0  1  1  0  1    2.0       1
15  1  0  0  0  1  0  0  1  0  1  1  0  1  1  0  1    2.0       1
4   1  1  0  0  1  0  0  1  0  1  1  0  1  0  0  1    5.0       1
3   1  1  0  0  1  0  0  1  0  1  1  0  1  1  0  1    6.0       1
2   1  1  0  0  0  1  1  1  1  1  1  0  0  1  0  1    8.0       2
9   1  1  0  0  0  1  1  1  1  1  1  0  0  1  0  1    8.0       1
17  0  1  0  0  1  0  0  0  0  1  0  0  1  1  0  1    8.0       1
10  0  1  0  0  1  1  1  1  1  0  1  0  1  1  0  1   14.0       2
8   1  0  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
1   1  0  0  0  1  0  0  1  0  1  0  0  1  1  1  1    0.0       1
2   0  0  0  1  0  1  1  1  0  0  1  0  0  1  1  1    0.0       1
0   1  0  0  0  1  0  0  1  0  1  0  0  1  0  1  1    1.0       1
3   0  0  0  0  0  1  1  1  0  1  0  0  1  1  1  0    1.0       1
4   0  0  0  1  0  1  1  0  1  1  0  0  0  1  1  1    1.0       1
5   1  1  0  1  0  1  1  0  1  0  0  0  0  1  0  1    1.0       1
6   0  0  0  0  1  0  1  1  0  1  1  0  1  0  1  1    1.0       1
7   0  0  0  1  0  0  1  1  0  1  1  0  1  1  1  0    1.0       1
8   1  0  0  0  1  0  0  1  0  0  1  1  0  0  1  1    1.0       1
9   1  0  0  1  1  0  0  1  0  1  0  0  1  0  1  1    2.0       1
10  1  0  0  0  1  0  0  1  0  0  0  1  1  0  1  1    2.0       1
11  0  0  0  0  1  0  0  1  0  1  0  1  1  0  1  1    2.0       1
12  1  1  0  0  1  0  0  1  0  0  0  1  1  1  1  0    2.0       1
13  0  1  0  0  1  0  0  1  0  1  0  0  1  1  1  1    2.0       1
14  0  1  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   1  1  0  0  1  0  0  1  1  1  0  0  0  0  1  1    1.0       1
1   1  1  0  0  1  0  0  1  0  1  0  0  1  1  1  0    1.0       1
2   1  1  0  1  0  0  0  1  0  0  0  1  1  0  0  1    2.0       1
3   1  0  0  0  0  1  0  1  1  1  0  0  1  1  1  0    2.0       1
4   1  1  1  0  0  0  1  0  1  0  0  0  1  1  0  0    2.0       1
5   0  1  0  0  0  0  1  1  0  0  1  1  1  1  0  1    2.0       1
6   0  0  1  0  0  1  1  0  1  1  1  0  0  1  1  0    2.0       1
7   1  0  1  1  1  0  0  0  1  0  1  0  0  1  0  0    3.0       1
8   0  0  1  1  1  0  1  0  0  0  1  1  0  1  0  0    3.0       1
9   0  0  0  1  1  1  0  0  0  0  1  0  0  1  1  0    4.0       1
10  0  1  1  0  0  1  0  1  1  0  0  0  1  1  0  1    4.0       1
11  1  1  1  0  0  1  0  1  1  0  0  0  1  1  0  0    5.0       1
12  0  0  0  0  0  0  0  0  1  1  1  1  1  0  1  0    5.0       1
13  1  0  1  0  1  0  1  1  0  0  1  0  1  0  1  0    5.0       1
14  0  0  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   0  0  1  0  0  1  1  1  0  1  1  0  0  1  1  0    1.0       1
1   1  0  1  0  1  0  1  0  0  1  1  0  0  0  1  1    1.0       1
2   1  1  0  1  1  0  1  1  0  1  0  0  0  0  0  1    1.0       1
3   0  0  0  0  1  1  1  0  0  1  0  0  1  1  0  1    1.0       1
4   1  0  0  0  1  1  1  0  0  1  0  0  1  1  0  1    1.0       1
5   1  1  0  0  1  0  1  0  0  0  1  0  1  0  1  0    2.0       1
6   0  0  1  0  1  0  1  0  1  1  0  1  0  1  0  1    2.0       1
7   0  0  0  1  0  1  1  1  0  0  0  0  0  1  1  1    2.0       1
8   1  0  1  0  1  1  0  0  0  1  1  0  0  1  1  1    3.0       1
9   1  1  0  0  1  0  1  1  1  0  1  0  0  0  1  1    3.0       1
10  1  0  1  1  0  0  0  1  1  0  0  0  0  1  0  0    4.0       1
11  0  0  0  1  1  1  1  1  0  1  1  0  0  1  0  0    4.0       1
12  0  0  0  0  1  1  1  1  0  0  1  1  0  0  0  0    5.0       1
13  1  0  1  0  0  1  1  1  1  0  0  1  0  1  0  0    5.0       1
14  0  1  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   1  1  0  0  0  0  1  0  0  1  1  0  1  0  1  1    0.0       1
1   1  0  0  0  0  0  0  1  0  0  1  1  1  0  1  1    1.0       1
2   0  1  0  0  0  0  0  1  1  0  0  1  1  1  1  1    1.0       1
3   1  1  0  0  0  1  1  1  0  0  1  0  1  0  0  0    2.0       1
4   1  0  0  0  1  0  0  1  0  0  1  1  1  0  1  1    2.0       1
5   0  1  1  0  0  1  1  1  0  0  0  1  0  1  1  0    2.0       1
6   1  0  0  0  0  1  1  1  0  1  0  0  0  0  1  1    2.0       1
7   1  0  0  1  0  1  1  0  0  1  0  0  0  0  1  1    3.0       1
8   1  1  0  0  0  1  0  0  0  1  1  1  1  0  0  1    4.0       1
9   0  0  1  0  0  1  0  0  0  0  1  0  1  1  1  0    4.0       1
10  0  0  0  0  0  0  1  0  0  1  1  1  1  0  1  0    4.0       1
11  1  1  0  0  0  0  1  1  1  0  0  1  0  0  0  0    4.0       1
12  1  1  0  1  1  1  0  1  0  0  1  0  0  0  1  0    5.0       1
13  1  0  0  1  0  0  0  1  0  1  0  1  1  0  0  0    5.0       1
14  0  0  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   0  0  1  1  0  0  0  0  1  1  1  0  0  1  0  1    3.0       1
1   1  1  0  0  1  0  0  1  0  1  1  0  0  1  1  1    3.0       1
2   1  1  0  1  0  0  1  0  1  0  1  1  0  0  0  0    3.0       1
3   1  0  1  1  0  1  0  0  0  1  1  0  0  0  1  0    3.0       1
4   1  0  0  1  1  1  0  0  0  1  0  0  0  0  0  1    4.0       1
5   1  1  1  0  0  0  0  0  1  1  1  1  0  0  0  1    4.0       1
6   0  0  0  0  1  0  1  0  1  1  1  0  1  0  0  0    5.0       1
7   1  0  1  0  1  0  0  1  0  0  1  1  1  0  0  1    5.0       1
8   0  1  1  0  0  1  0  0  0  0  1  0  0  0  1  1    5.0       1
9   0  0  1  1  0  1  0  0  0  0  0  0  1  1  1  0    5.0       1
10  0  1  0  0  0  1  1  1  1  0  1  0  1  0  1  0    5.0       1
11  0  1  0  0  0  0  1  1  0  1  1  1  0  0  0  1    6.0       1
12  0  1  0  0  1  0  0  0  0  0  1  1  1  0  0  1    6.0       1
13  0  0  0  1  0  1  0  0  0  1  0  1  1  0  1  0    6.0       1
14  1  0  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   1  1  0  0  0  1  1  1  0  0  0  1  0  1  1  0    1.0       1
1   0  1  0  0  1  0  1  1  0  0  0  1  0  1  1  0    1.0       1
2   1  1  0  1  0  0  1  0  0  1  1  1  0  0  0  1    1.0       1
3   1  1  0  0  1  0  1  1  0  0  0  1  0  1  0  0    1.0       1
4   1  1  1  0  0  0  1  1  0  0  0  1  0  1  1  0    1.0       1
5   0  0  1  1  0  1  0  1  1  0  0  0  0  1  1  1    2.0       1
6   1  1  1  0  0  1  1  1  0  0  0  1  0  1  0  0    2.0       1
7   0  1  1  0  1  1  1  0  0  0  0  0  0  1  0  0    4.0       1
8   1  0  1  1  1  0  0  0  0  0  1  0  0  1  0  0    4.0       1
9   1  0  0  0  0  1  0  1  1  0  0  0  0  1  1  0    5.0       1
10  0  1  1  1  0  1  0  1  0  0  1  0  0  0  1  0    5.0       1
11  1  0  1  0  0  1  1  0  0  1  1  1  0  0  1  0    5.0       1
12  1  0  1  0  0  1  1  1  0  0  1  1  0  0  1  1    6.0       1
13  1  0  0  0  0  1  1  1  1  0  0  0  0  0  1  0    6.0       1
14  0  0  

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0   1  0  1  0  0  0  0  1  0  1  0  0  1  1  1  1    0.0       1
1   0  1  0  1  0  0  0  0  0  1  0  1  1  1  1  1    1.0       1
2   1  1  0  1  0  0  1  0  0  0  0  1  0  1  1  1    2.0       1
3   0  0  1  0  1  0  1  0  0  1  1  1  0  0  0  1    2.0       1
4   0  1  0  0  0  0  0  1  0  0  0  1  1  1  1  1    2.0       1
5   0  0  1  1  0  0  0  1  0  1  0  1  0  1  1  0    2.0       1
6   0  0  1  1  1  0  0  0  0  1  0  0  1  0  1  1    3.0       1
7   1  1  0  1  0  0  1  1  0  1  1  1  0  0  0  0    4.0       1
8   1  0  1  1  0  0  1  0  1  0  0  0  1  1  0  0    5.0       1
9   0  0  0  0  1  1  0  0  0  0  0  0  1  1  1  1    5.0       1
10  1  0  1  0  1  0  1  1  0  1  1  0  1  0  0  0    5.0       1
11  0  0  1  0  1  1  1  0  1  0  0  0  0  0  0  1    5.0       1
12  0  0  1  0  1  0  1  0  0  1  0  1  1  1  1  0    5.0       1
13  0  0  1  1  0  1  1  1  0  0  0  1  0  1  1  1    6.0       1
14  0  1  

([[1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1],
  [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1],
  [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1],
  [1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1],
  [0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1],
  [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1],
  [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1],
  [1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1],
  [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
  [0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0],
  [0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1],
  [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
  [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1],
  [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1],
  [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1],
  [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
  [1, 1, 0, 

### ここまでのまとめ
- max_chain_strengthは増やしたほうがいい
- chain_break_fraction=True
- num_readsも増やしたほうが良い

In [25]:
def test_validity():
    df1 = pd.read_csv('../../input/ost16.csv', sep=',',index_col=0)
    df2 = pd.read_csv('../../input/ost16.csv', sep=',',index_col=0)
    new_y = np.array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1])
    df2['Y'] = new_y
    t_list1 = calc_marginals(df1)
    t_list2 = calc_marginals(df2)
    print(t_list1)
    print(t_list2)
    assert np.all(t_list1[[0,2,3]] == t_list2[[0,2,3]]) 
test_validity()

[8 6 4 4]
[8 6 4 4]


In [72]:
def time_measurement(df):
    sum_time = 0
    annealing_time = 20
    for t1 in range(0, sum(df['LI'])+1):
        timeit_repeat = timeit.repeat("make_Hamiltonian(df, t1)", number=1, repeat=1, globals={"make_Hamiltonian": make_Hamiltonian, "df": df, "t1": t1})
        sum_time += timeit_repeat[0] + annealing_time
    return sum_time

In [73]:
#==========
#テストコード
#==========
def test_find_valid_y():
    df = pd.read_csv('../../input/ost6.csv', sep=',', index_col=0)
    true_t1 = sum(df['Y'] * df['LI'])
    valid_y_list, valid_y_num = find_valid_y(df,  num_reads=20)
    print(valid_y_list, valid_y_num)
    assert valid_y_num[true_t1] > 0
    
test_find_valid_y()

{0: [], 1: [], 2: [[1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0]], 3: [], 4: [], 5: [], 6: []} {0: 0, 1: 0, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0}


In [64]:
def test_validity():
    df1 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    df2 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    new_y = np.array([1, 0, 0, 0, 0, 1])
    df2['Y'] = new_y
    t_list1 = calc_marginals(df1)
    t_list2 = calc_marginals(df2)
    print(t_list1)
    print(t_list2)
    assert np.all(t_list1[[0,2,3]] == t_list2[[0,2,3]]) 

#test_validity()

[2 2 0 1]
[2 2 0 1]
